In [0]:
%sql 
--Creating a database connection in unity catalog of databricks

CREATE CONNECTION gcp_mysql_conn_we3
TYPE mysql
OPTIONS (host '34.123.166.158',
port '3306',
user 'devuser',
password 'Inceptez@123');


In [0]:
%sql
CREATE FOREIGN CATALOG gcp_mysql_fc_we3
USING CONNECTION gcp_mysql_conn_we3

In [0]:
%sql
DESCRIBE CATALOG gcp_mysql_fc_we3

In [0]:
%sql
SHOW CATALOGS;

In [0]:
%sql
SELECT * FROM gcp_mysql_fc_we3.logistics.shipments1;

In [0]:
%sql
-- Bronze incremental ingestion from database using foreign catalog connection
-- First load will be historical load


CREATE TABLE IF NOT EXISTS soundhar_catalog.soundhar_schema.shipment_his_ingestion 
USING DELTA
AS SELECT 
  shipment_id,
  first_name,
  last_name,
  age,
  role,
  city,
  updated_at  
FROM gcp_mysql_fc_we3.logistics.shipments1;


In [0]:
%sql
SELECT * FROM soundhar_catalog.soundhar_schema.shipment_his_ingestion;

In [0]:
%sql 
-- Implementing SCD Type2
INSERT INTO soundhar_catalog.soundhar_schema.shipment_his_ingestion 
SELECT 
  shipment_id,
  first_name,
  last_name,
  age,
  role,
  city,
  updated_at
FROM gcp_mysql_fc_we3.logistics.shipments1
WHERE updated_at > ( SELECT COALESCE(MAX(updated_at),'1970-01-01')
FROM soundhar_catalog.soundhar_schema.shipment_his_ingestion);



In [0]:
%sql 
SELECT * FROM soundhar_catalog.soundhar_schema.shipment_his_ingestion

In [0]:
%sql

SELECT *, row_number() over (partition by shipment_id order by updated_at desc) as rn
from soundhar_catalog.soundhar_schema.shipment_his_ingestion
qualify rn = 1